<a href="https://colab.research.google.com/github/solangecdh3/assignment_1/blob/main/hackathons/conditional_loop_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manipulating Data Hackathon

Ok first things first... let's assign y'all to groups!

In [1]:
import random

# Set the random seed for reproducibility
random.seed(4)

students = [
    "Adam", "Andrew", "Cameron", "Liam",
    "Lindsay", "Mason", "Solange", "Simon"
]

random.shuffle(students) # Randomly shuffles the list of students in place

# Create groups of 2
for i in range(0, len(students), 2):
    group = students[i:i+2]
    print(f"Group: {group}")

Group: ['Andrew', 'Solange']
Group: ['Mason', 'Lindsay']
Group: ['Simon', 'Adam']
Group: ['Cameron', 'Liam']


Ok now let's unzip in the data.  

In [14]:
# --- Setup ---
import os, glob, zipfile
from pathlib import Path
import pandas as pd

BASE = Path("hackathon_dataset")
ZIP  = Path("hackathon_dataset.zip")

# Unzip if needed
if not BASE.exists() and ZIP.exists():
    import zipfile
    with zipfile.ZipFile(ZIP, "r") as zf:
        zf.extractall("")

print("Data folder exists:", BASE.exists())


Data folder exists: True


## Task 1 — Load & Combine

Ok, your first task is to load and combine the data.  We want to:

1) Read all 'measurements_2024-??.csv' and handle schema drift with simple if statements
2) Add 'source_file', and concat with union of columns

In [15]:
# --- Task 1: Load & Combine (union of columns, source_file) ---
# Read all 'measurements_2024-??.csv', handle schema drift with simple if statements,
# add 'source_file', and concat with union of columns.

## Here's a hint: you can use `if "String" in df.columns` and a conditional!
import pandas as pd
from pathlib import Path

data_folder = BASE / "data"
files = sorted((data_folder).glob("measurements*.csv")) ## fill this in
all_dfs = []

for f in files:
    df_month = pd.read_csv(f)

    # TO DO: Handle schema drift: tempC -> temp_c

if "tempC" in df_month.columns:
        df_month["temp_c"] = df_month["tempC"]

    # TO DO: Handle schema drift: sensorID -> sensor_id

if "sensorID" in df_month.columns:
        df_month["sensor_id"] = df_month["sensorID"]
    # TO DO: Add provenance column

df_month["source_file"] = f.name

all_dfs.append(df_month)

# Union-concat: sort=False keeps all columns encountered across files
df_all = pd.concat(all_dfs, ignore_index=True, sort=False)
print("Combined shape:", df_all.shape)
df_all.head()


Combined shape: (237, 6)


,date,sensor_id,temp_c,status,notes,source_file
0,2024-12-26,S007,22.29,WARN,manual check,measurements_2024-12.csv
1,2024-12-03,S015,7.07,ERROR,recalibrated,measurements_2024-12.csv
2,2024-12-27,S017,-1.51,NaN,low battery,measurements_2024-12.csv
3,2024-12-02,S008,8.19,OK,maintenance window,measurements_2024-12.csv
4,2024-12-16,S002,4.95,Ok,NaN,measurements_2024-12.csv


## Task 2 — Normalize + Filter + Flag Suspicious Temps

Ok now that we've done that, let's create a create 'status_norm' with loops and basic string methods.  I'll provide you that code!

Then let's drop rows where status missing/empty or equals 'error' (case-insensitive).

Also, let's create 'suspicious_temp' when `temp_c < -30 or > 50` (row-wise with loops).

In [19]:
# --- Task 2: Normalize 'status' and filter + flag suspicious temps ---
# We'll create 'status_norm' with loops and basic string methods.
# Then: drop rows where status missing/empty or equals 'error' (case-insensitive).
# Also create 'suspicious_temp' when temp_c < -30 or > 50 (row-wise with loops).

import math

# 1) Normalize status -> 'status_norm'
status_norm = []
for i in range(len(df_all)):
    val = df_all.loc[i, "status"] if "status" in df_all.columns else ""
    # treat NaN as empty string
    if pd.isna(val):
        val = ""
    val = str(val).strip().lower()
    status_norm.append(val)

df_all["status_norm"] = status_norm

# 2) Flag suspicious temps
suspicious = []
for i in range(len(df_all)):
    if "temp_c" in df_all.columns:
        t = df_all.loc[i, "temp_c"]
        # handle NaN
        if pd.isna(t):
            suspicious.append(False)
        else:
            # Check if temperature is suspicious (< -30 or > 50)
            if t < -30 or t > 50:
                suspicious.append(True)
            else:
                suspicious.append(False)
    else:
        suspicious.append(False)

df_all["suspicious_temp"] = suspicious

# 3) Filter out missing/empty status OR 'error'
keep = []
for i in range(len(df_all)):
    s = df_all.loc[i, "status_norm"]
    # Keep rows where status is not empty AND not 'error'
    if s != "" and s != "error":
        keep.append(True)
    else:
        keep.append(False)

df_clean = df_all.loc[keep].copy()
print("After status cleaning:", df_clean.shape)
df_clean.head()


After status cleaning: (175, 8)


,date,sensor_id,temp_c,status,notes,source_file,status_norm,suspicious_temp
0,2024-12-26,S007,22.29,WARN,manual check,measurements_2024-12.csv,warn,False
3,2024-12-02,S008,8.19,OK,maintenance window,measurements_2024-12.csv,ok,False
4,2024-12-16,S002,4.95,Ok,NaN,measurements_2024-12.csv,ok,False
5,2024-12-03,S013,3.55,OK,NaN,measurements_2024-12.csv,ok,False
6,2024-12-24,S012,-2.42,WARN,NaN,measurements_2024-12.csv,warn,False


## Task 3 — Validate Rows (loops only)

Ok now let's try to make sure all the values are there... I'll provide the code for `is_data` (and it's a try statement! that's new!) as well as `is_sensor_ok`, but you do the rest!

In [27]:
# --- Task 3: Validation ---
# We create: is_date, is_sensor_ok, is_temp_ok, is_status_ok, then 'is_valid' with loops.

# 1) is_date: try parsing each, if fails -> False
is_date = []
for i in range(len(df_clean)):
    try:
        pd.to_datetime(df_clean.iloc[i]["date"], errors="raise")
        is_date.append(True)
    except Exception:
        is_date.append(False)

# 2) is_sensor_ok: non-empty string in 'sensor_id'
is_sensor_ok = []
for i in range(len(df_clean)):
    if "sensor_id" in df_clean.columns:
        sid = df_clean.iloc[i]["sensor_id"]
    else:
        sid = ""
    if pd.isna(sid):
        is_sensor_ok.append(False)
    else:
        sid = str(sid)
        if len(sid) > 0:
            is_sensor_ok.append(True)
        else:
            is_sensor_ok.append(False)

# 3) is_temp_ok: -50 <= temp_c <= 60 (NaN -> False)
is_temp_ok = []
for i in range(len(df_clean)):
    if "temp_c" in df_clean.columns:
        temp = df_clean.iloc[i]["temp_c"]
    else:
        temp = float('nan')

    if pd.isna(temp):
        is_temp_ok.append(False)
    else:
        if temp >= -50 and temp <= 60:
            is_temp_ok.append(True)
        else:
            is_temp_ok.append(False)

# 4) is_status_ok: in {'ok','warn','error'}
is_status_ok = []
valid_statuses = {'ok', 'warn', 'error'}
for i in range(len(df_clean)):
    if "status_norm" in df_clean.columns:
        status_val = df_clean.iloc[i]["status_norm"]
    else:
        status_val = ""

    if pd.isna(status_val):
        is_status_ok.append(False)
    else:
        if status_val in valid_statuses:
            is_status_ok.append(True)
        else:
            is_status_ok.append(False)

df_clean["is_date"] = is_date
df_clean["is_sensor_ok"] = is_sensor_ok
df_clean["is_temp_ok"] = is_temp_ok
df_clean["is_status_ok"] = is_status_ok

# 5) is_valid = all four True
is_valid = []
for i in range(len(df_clean)):
    if (df_clean.iloc[i]["is_date"] and
        df_clean.iloc[i]["is_sensor_ok"] and
        df_clean.iloc[i]["is_temp_ok"] and
        df_clean.iloc[i]["is_status_ok"]):
        is_valid.append(True)
    else:
        is_valid.append(False)

df_clean["is_valid"] = is_valid

print("Valid fraction:", round(float(sum(is_valid))/len(is_valid), 3))
df_clean[["is_date","is_sensor_ok","is_temp_ok","is_status_ok","is_valid"]].head()

Valid fraction: 0.954


,is_date,is_sensor_ok,is_temp_ok,is_status_ok,is_valid
0,True,True,True,True,True
3,True,True,True,True,True
4,True,True,True,True,True
5,True,True,True,True,True
6,True,True,True,True,True


## Task 4 — Logs Parsing

Ok now for the last thing, you try the whole thing again but on the `logs` data.  So your goal is to count how many rows per month have status 'ok' (case-insensitive, strip spaces).

You got this!!

In [31]:
# --- Task 4: Logs parsing (months 01..09) with loops ---
# Goal: count how many rows per month have status 'ok' (case-insensitive, strip spaces).




## Task 5 — Save Outputs

And finally, save that out!  You'll submit (in canvas) a `measurements_2024_clean_basic.csv` and `logs_ok_counts_basic.csv' (one for each group).

In [30]:
# --- Task 5: Save outputs ---
out_dir = BASE / "outputs_basic"
out_dir.mkdir(exist_ok=True)
df_clean.to_csv(out_dir/"measurements_2024_clean_basic.csv", index=False)
ok_counts.to_csv(out_dir/"logs_ok_counts_basic.csv", index=False)
print("Wrote outputs to:", out_dir)


Wrote outputs to: hackathon_dataset/outputs_basic
